In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
warnings.simplefilter('ignore')

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

### ANALISE E FILTRAGEM DOS DADOS

In [ ]:
df_info = pd.read_csv('/content/drive/MyDrive/Mackenzie/Projeto Aplicado 3/Course_info.csv')

In [ ]:
df_info.head()

In [ ]:
df_info.shape

(209734, 20)

In [ ]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209734 entries, 0 to 209733
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  209734 non-null  float64
 1   title               209734 non-null  object 
 2   is_paid             209734 non-null  bool   
 3   price               209734 non-null  float64
 4   headline            209703 non-null  object 
 5   num_subscribers     209734 non-null  float64
 6   avg_rating          209734 non-null  float64
 7   num_reviews         209734 non-null  float64
 8   num_comments        209734 non-null  float64
 9   num_lectures        209734 non-null  float64
 10  content_length_min  209734 non-null  float64
 11  published_time      209734 non-null  object 
 12  last_update_date    209597 non-null  object 
 13  category            209734 non-null  object 
 14  subcategory         209734 non-null  object 
 15  topic               208776 non-nul

In [ ]:
df_info.columns

Index(['id', 'title', 'is_paid', 'price', 'headline', 'num_subscribers',
       'avg_rating', 'num_reviews', 'num_comments', 'num_lectures',
       'content_length_min', 'published_time', 'last_update_date', 'category',
       'subcategory', 'topic', 'language', 'course_url', 'instructor_name',
       'instructor_url'],
      dtype='object')

In [ ]:
df_info['language'].unique()

array(['English', 'Spanish', 'Turkish', 'Simplified Chinese', 'Arabic',
       'Portuguese', 'Italian', 'Serbian', 'Afrikaans', 'French',
       'Slovak', 'Japanese', 'Hebrew', 'Traditional Chinese', 'Estonian',
       'German', 'Russian', 'Finnish', 'Dutch', 'Hungarian', 'Swedish',
       'Norwegian', 'Thai', 'Bulgarian', 'Polish', 'Urdu', 'Croatian',
       'Marathi', 'Danish', 'Greek', 'Hindi', 'Indonesian', 'Azeri',
       'Vietnamese', 'Bengali', 'Persian', 'Malay', 'Korean', 'Filipino',
       'Romanian', 'Catalan', 'Czech', 'Albanian', 'Telugu', 'Latvian',
       'Ukrainian', 'Tamil', 'Somali', 'Burmese', 'Kazakh', 'Tatar',
       'Kurdish', 'Aymara', 'Malayalam', 'Uzbek', 'Georgian',
       'Lithuanian', 'Gujarati', 'Pashto', 'Haitian', 'Nepali',
       'Armenian', 'Punjabi', 'Swahili', 'Mongolian', 'Slovenian', 'Zulu',
       'Kannada', 'Guaraní', 'Quechua', 'Macedonian', 'Khmer', 'Frisian',
       'Welsh', 'Irish', 'Basque', 'Xhosa', 'Tajik', 'Faroese'],
      dtype=object)

In [ ]:
df = df_info[['id', 'title', 'price', 'avg_rating', 'headline']].copy()

In [ ]:
df = df_info[df_info['language'].isin(['English', 'Spanish', 'Portuguese'])].copy()

In [ ]:
df.head()

In [ ]:
df.shape

(159820, 20)

In [ ]:
df = df_info[(df_info['price'] <= 20) & (df_info['avg_rating'] >= 4.5)].copy()

In [ ]:
df.shape

(26379, 20)

In [ ]:
df = df.dropna(subset=['headline']).copy()

In [ ]:
df.shape

(26373, 20)

In [ ]:
df_final = df.copy()

In [ ]:
df_final['headline'] = df_final['headline'].str.lower()

In [ ]:
df_final.to_csv('df_final.csv', index=False)

# TESTE DO MODELO

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Mackenzie/Projeto Aplicado 3/df_final.csv')

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
headlines = df['headline'].tolist()

In [ ]:
# embeddings = model.encode(headlines, show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/825 [00:00<?, ?it/s]

In [ ]:
embeddings_path = '/content/drive/MyDrive/Mackenzie/Projeto Aplicado 3/udemy_courses_headline_embeddings.npy'

In [ ]:
# np.save(embeddings_path, embeddings)

# Recommendation - Tests

---



In [ ]:
embeddings = np.load(embeddings_path)

In [ ]:
df = df[['id', 'title', 'price', 'avg_rating', 'course_url', 'instructor_name', 'headline']].copy()

In [ ]:
def recommendation(skills, top_k=100, results=10):
    query = ', '.join(skills)
    query_embedding = model.encode([query])

    similarities = cosine_similarity(query_embedding, embeddings)[0]

    top_k_indices = np.argsort(similarities)[-top_k:][::-1]

    candidate_course = df.iloc[top_k_indices].copy()
    candidate_course['similarity'] = similarities[top_k_indices]
    candidate_course = candidate_course.sort_values('similarity', ascending=False)

    return candidate_course.head(results)

In [ ]:
skills = ['data science', 'python', 'machine learning']

In [ ]:
recommendation(skills, 50, 20)